In [16]:
import cv2   
import numpy as np

#capturing video through webcam
cap=cv2.VideoCapture(0)
cap.set(3,1024)
cap.set(4,760)


# define the lower and upper boundaries of the colors.
lower = {'red':(166, 84, 141), 'green':(66, 122, 129), 'blue':(99,115,150), 'yellow':(23, 59, 119), 'orange':(0, 50, 80)} #assign new item lower['blue'] = (93, 10, 0)
upper = {'red':(186,255,255), 'green':(86,255,255), 'blue':(110,255,255), 'yellow':(54,255,255), 'orange':(20,255,255)}
 
# define standard colors for circle.
colors = {'red':(0,0,255), 'green':(0,255,0), 'blue':(255,0,0), 'yellow':(0, 255, 217), 'orange':(0,140,255)}
 


while(1):
    # grab the frame
    (grabbed, frame) = cap.read()
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #for each color in dictionary check object in frame
    for key, value in upper.items():
        kernel = np.ones((9,9),np.uint8)
        mask = cv2.inRange(hsv, lower[key], upper[key])
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[-2]
        center = None
       
        # only proceed if at least one contour was found
        if len(cnts) > 0:
            #compute the minimum enclosing circle and centroid
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
       
            # only proceed if the radius meets a minimum size.
            if radius > 0.5:
                # draw the circle and centroid on the frame,
                cv2.circle(frame, (int(x), int(y)), int(radius), colors[key], 2)
                cv2.circle(frame, center, 3, colors[key],-1)
                cv2.putText(frame,key, (int(x-radius),int(y-radius)), cv2.FONT_HERSHEY_SIMPLEX, 0.6,colors[key],2)
                cv2.putText(frame,"("+str(center[0])+","+str(center[1])+")",(center[0]+10,center[1]+15), cv2.FONT_HERSHEY_SIMPLEX,0.6,colors[key],1)
                
    # show the frame to our screen
    cv2.imshow("Frame", frame)
    
    #If escape key is press window will be closed.
    if cv2.waitKey(1)==27:
        cap.release()
        cv2.destroyAllWindows()
        break 
    #close any open windows